In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Watches Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Watches_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [2]:
#Create DataFrame - Perform Preliminary Cleaning

#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(960872, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|Invicta Women's 1...|         Watches|          5|            0|          0|                Y|
|   14661224| RKH8BNC3L5DLF|B00D3RGO20|Kenneth Cole New ...|         Watches|          5|            0|          0|                Y|
|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|Ritche 22mm Black...|         Watches|          2|            1|          1|                Y|
|    7211452|R31U3UH5AZ42LL|B000EQS1JW|Citizen Men's BM8...|         Watches|          5|            0|          0|                Y|
|   12733322|R2SV659OUJ945Y|B00A6GFD7S|Orient ER27009B M...|  

In [4]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))
     

(831415, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|Invicta Women's 1...|         Watches|          5|            0|          0|
|   14661224| RKH8BNC3L5DLF|B00D3RGO20|Kenneth Cole New ...|         Watches|          5|            0|          0|
|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|Ritche 22mm Black...|         Watches|          2|            1|          1|
|    7211452|R31U3UH5AZ42LL|B000EQS1JW|Citizen Men's BM8...|         Watches|          5|            0|          0|
|   12733322|R2SV659OUJ945Y|B00A6GFD7S|Orient ER27009B M...|         Watches|          4|            0|          0|
|    6576411| RA51CP8TR5A2L|B00EYSOSE8|Casio Men's GW-94...|         Wat

In [6]:
#Create Analysis-Specific DFs/Tables
#Perform Analysis-Specific Transforms
#Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|    3653882|         Watches|
|   14661224|         Watches|
|   27324930|         Watches|
|    7211452|         Watches|
|   12733322|         Watches|
|    6576411|         Watches|
|   11811565|         Watches|
|   45925069|         Watches|
|   44751341|         Watches|
|    9962330|         Watches|
|   51330346|         Watches|
|    4201739|         Watches|
|   26339765|         Watches|
|    2692576|         Watches|
|   44713366|         Watches|
|   32778769|         Watches|
|   27258523|         Watches|
|   42646538|         Watches|
|   46017899|         Watches|
|   37192375|         Watches|
+-----------+----------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')


#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'watches')
#2c Check results
segment_df.show()

+-----------+-------+
|customer_id|watches|
+-----------+-------+
|    1567510|      1|
|   19502021|      1|
|   12819130|      1|
|   35329257|      2|
|     108460|      1|
|    5453476|      1|
|   29913055|      1|
|   30717305|      1|
|    1570030|      1|
|   44178035|      1|
|   26079415|      2|
|   14230926|      1|
|   43478048|      2|
|   43694941|      1|
|   12318815|      3|
|   13731855|      1|
|     740134|      1|
|   41956754|      1|
|   20324070|      3|
|   10767388|      2|
+-----------+-------+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- watches: long (nullable = false)

631959


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2


#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('watches', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-------+
|customer_id|watches|
+-----------+-------+
|   40765068|    127|
|    5072809|    105|
|   42416004|     86|
|   22962882|     66|
|   44191290|     64|
|   21375608|     59|
|   25160623|     59|
|   45112699|     59|
|    1389173|     58|
|    8887467|     55|
|   28244369|     55|
|    5956842|     53|
|   18840148|     52|
|   52581394|     52|
|   52716989|     52|
|   47769852|     51|
|   20631789|     49|
|   21148390|     48|
|   31060423|     46|
|   14726863|     45|
+-----------+-------+
only showing top 20 rows

100000


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}


In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres


# Write segment table to Postgres/RDS
# 23 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='watches_segment', mode=mode, properties=config)
     

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM watches_segment;

# Check 10 Rows of Segment Table - SELECT * FROM watches_segment LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful
